In [12]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [13]:
ee.Initialize()

In [14]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('../Process_2_GEE_Python_Classification/')

# import the temporal_consistency_check class
from BackGround_modules.Class_5_Temporal_consistency_check import Temporal_consistency_check

### Define basic parameters

In [15]:
# import the tranning points
Traning_pts = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Sample_Points/Sentinel_2017_2019")

In [16]:
# define the names of each period
year_range = list(f'{i[0]}_{i[1]}' for i in zip(range(1990,2018,3),range(1992,2020,3)))

### Import Classified_random_img

In [17]:
# get the classified_random_imgs and sum them up for each period
Classified_Landsat_1990_2019 = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Control_{year}")
                                  .sum().gte(8).set('name',year)   for year in year_range]

Classified_Sentinel_2014_2019 = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Sentinel_Landsat_{year}")
                                  .sum().gte(8).set('name',year)   for year in year_range[-2:]]

# combine classification img together
Classified_imgs = Classified_Landsat_1990_2019[:-2] + Classified_Sentinel_2014_2019

In [18]:
Iter_temporal_check_instaces = Temporal_consistency_check(Classified_imgs,3,10).Iterate_the_check()

Check length is    ---> 3
Check weights is   ---> [3, 1, 1]
Check threshold is ---> 4


In [19]:
Map = geemap.Map()
Map.setCenter(115.4508, 35.2492,10)

year_idx = 3

Map.add_basemap('HYBRID')
Map.addLayer(Classified_imgs[year_idx] ,{'min':0,'max':1},'origin')
Map.addLayer(Iter_temporal_check_instaces[0][year_idx] ,{'min':0,'max':1},'Iter_0')
Map.addLayer(Iter_temporal_check_instaces[1][year_idx] ,{'min':0,'max':1},'Iter_1')
Map.addLayer(Iter_temporal_check_instaces[2][year_idx] ,{'min':0,'max':1},'Iter_2')

Map

Map(center=[35.2492, 115.4508], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…